In [ ]:
import pickle as pkl
import pandas as pd
from pandas import DataFrame as df
from collections import defaultdict
import scipy.stats as stats
import numpy as np
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
def plot_cdf(l):
    fig = plt.figure()
    l.sort()
    p = np.arange(0,1,1/float(len(l)))
    plt.plot(l,p)
    plt.show()

def plot_hints(x, y_hint, y_nohint):
    plt.plot(x, y_hint, label='hint')
    plt.plot(x, y_nohint, label='no hint')
    plt.legend(loc=4)
    plt.xticks(x)

def get_hint_count():
    hint_count_all = defaultdict(list)
    hint_count_none = defaultdict(list)
    hint_count_uni = defaultdict(list)
    hint_count_exp = defaultdict(list)
    hint_count_con = defaultdict(list)
    for ind, row in HW_data.iterrows():
        col_ind = 0
        row_fine = row.fillna(-111)
        single_count_all = defaultdict(int)
        single_count_non = defaultdict(int)
        single_count_uni = defaultdict(int)
        single_count_exp = defaultdict(int)
        single_count_con = defaultdict(int)
        for ro in row_fine:
            all_h = 0
            non_h = 0
            exp_h = 0
            uni_h = 0
            con_h = 0
            if col_ind >= len(row)-1:
                continue
            title = cols[col_ind]
            if int(title[0]) < 4:
                col_ind += 1
                continue
            pro_id = (title[0],title[1])
            if ro != -111:
                hint_c = [s[3]=='Hint' for s in ro]
                all_h = sum(hint_c)
                if all_h > 0:
                    for r in ro:
                        if r[3]!='Hint':
                            continue
                        if r[2][0] == " ":
                            con_h += 1
                        elif r[2] == "no hint":
                            non_h += 1
                        elif r[2][:6] == "Please":
                            exp_h += 1
                        elif r[2][:6] != "Please":
                            uni_h += 1
                        else:
                            print "Error! Other hints not captured"
                            print r[2]
            single_count_all[pro_id] += all_h
            single_count_non[pro_id] += non_h
            single_count_uni[pro_id] += uni_h
            single_count_exp[pro_id] += exp_h
            single_count_con[pro_id] += con_h
            col_ind += 1
        for p in problems:
            if int(p[0]) < 4:
                continue
            hint_count_all[p].append(single_count_all[p])
            hint_count_none[p].append(single_count_non[p])
            hint_count_uni[p].append(single_count_uni[p])
            hint_count_exp[p].append(single_count_exp[p])
            hint_count_con[p].append(single_count_con[p])
    return hint_count_all, hint_count_none, hint_count_uni, hint_count_exp, hint_count_con


def add_to_df(fea_name, fea_df, df):
    for k in fea_df.keys():
        df[(k[0], k[1], fea_name)] = fea_df[k]

def stats_between_dfs(h_df, noh_df, final_col, print_enabled=True):
    h = len(h_df)
    no_h = len(noh_df)
    if h == 0 or no_h == 0:
        print "ERROR: Empty dataframe passed in..."
        return
    hint_mean = h_df[final_col].mean()
    nh_mean = noh_df[final_col].mean()
    hint_std = h_df[final_col].std()
    nh_std = noh_df[final_col].std()
    
    if(print_enabled):
        print h
        print no_h
        print 'with_hint: ', h/(h+no_h*1.)
        print 'with_no_hint: ', no_h/(h+no_h*1.)

        print '\n mean'
        print hint_mean
        print nh_mean

        print '\n std'
        print hint_std
        print nh_std

        print '\n ttest'
    return stats.ttest_ind_from_stats(hint_mean,hint_std, h, nh_mean, nh_std, no_h)

def add_assign(assign, df):
    stu = df.username.tolist()
    assign = assign.set_index('USERNAME')
    filtered_assign = assign.ix[stu]
    df[('4','assign')] = filtered_assign['WEEK4']
    df[('5','assign')] = filtered_assign['WEEK5']
    df[('6','assign')] = filtered_assign['WEEK6']
    df[('7','assign')] = filtered_assign['WEEK7']
    df[('8','assign')] = filtered_assign['WEEK8']

def get_time_diff():
    tim_diff = defaultdict(list)
    for ind, row in HW_data.iterrows():
        col_ind = 0
        row_fine = row.fillna(-111)
        single_count = defaultdict(int)
        for ro in row_fine:
            if col_ind >= len(row)-1:
                continue
            title = cols[col_ind]
            pro_id = (title[0],title[1])
            if ro == -111:
                single_count[pro_id] += 0
            elif len(ro) > 0:
                t1 = pd.to_datetime(ro[0][0])
                t2 = pd.to_datetime(ro[-1][0])
                tmp = t2-t1
                single_count[pro_id] += tmp.seconds
            col_ind += 1
        for p in problems:
            tim_diff[p].append(single_count[p])
    return tim_diff

def get_tries():
    tries_count = defaultdict(list)
    for ind, row in HW_data.iterrows():
        col_ind = 0
        row_fine = row.fillna(-111)
        single_count = defaultdict(int)
        for ro in row_fine:
            if col_ind >= len(row)-1:
                continue
            title = cols[col_ind]
            pro_id = (title[0],title[1])
            if ro == -111:
                single_count[pro_id] += 0
            elif len(ro) >= 0:# and ro[-1][2] == '1':
                tries_c = sum([r[3]!='Hint' for r in ro])
                single_count[pro_id] += tries_c
            col_ind += 1
        for p in problems:
            tries_count[p].append(single_count[p])
    return tries_count

def get_time_spent():
    tim_diff = defaultdict(list)
    for ind, row in HW_data.iterrows():
        col_ind = 0
        row_fine = row.fillna(-111)
        single_count = defaultdict(int)
        for ro in row_fine:
            if col_ind >= len(row)-1:
                continue
            title = cols[col_ind]
            pro_id = (title[0],title[1])
            if ro == -111:
                single_count[pro_id] += 0
            elif len(ro) > 0:
                for r in range(len(ro)-1):
                    t1 = pd.to_datetime(ro[r][0])
                    t2 = pd.to_datetime(ro[r+1][0])
                    tmp = t2-t1
                    if tmp.seconds//60 < 10:
                        single_count[pro_id] += tmp.seconds
            col_ind += 1
        for p in problems:
            tim_diff[p].append(single_count[p])
    return tim_diff

def fea_p_val(fea_name, df):
    p_vals = []
    pro_list = []
    h = []
    nh = []
    pros_stat = []
    for p in sorted(problems):
        if int(p[0]) < 4:
            continue
        pn = (p[0], p[1], 'hint_count')
        h_df = df[(df[pn] > 0)]
        nh_df = df[(df[pn] == 0) & (df[(p[0],'assign')] == '0')]
        if len(h_df) == 0 or len(nh_df) == 0:
            continue
        pros_stat.append(p)
        st = stats_between_dfs(h_df, nh_df, (p[0],p[1],fea_name), False)
        if st and st[1] == st[1]:
            p_vals.append(st[1])
            pro_list.append(p)
            h.append(h_df[p[0],p[1],fea_name].mean())
            nh.append(nh_df[p[0],p[1],fea_name].mean())
    return p_vals, pro_list, h, nh

def fig_setting():
    plt.figure(figsize=(10,5))
    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    return plt

def find_ave(df):
    ### Average tries for students with no hint
    nh_ave = {}  #{pro_id: average}
    ### Average tries for students with no click
    h_nc_ave = {}
    ### Average tries for students with hints
    h_ave = {}

    for p in sorted(problems):
        if int(p[0]) < 4:
            continue
        pn = (p[0], p[1], 'hint_count')

        nh_df = df[(df[(p[0],'assign')] == '0') & (df[(p[0],p[1],'tries')] > 0)]
        if len(nh_df) != 0:
            nh_ave[(p[0],p[1])] = nh_df[(p[0],p[1],'tries')].mean()

        nc_df = df[(df[(p[0],'assign')] != '0') & (df[pn] == 0) & (df[(p[0],p[1],'tries')] > 0)]
        if len(nc_df) != 0:
            h_nc_ave[(p[0],p[1])] = nc_df[(p[0],p[1],'tries')].mean()

        h_df = df[df[pn] > 0 & (df[(p[0],p[1],'tries')] > 0)]
        if len(h_df) != 0:
            h_ave[(p[0],p[1])] = h_df[(p[0],p[1],'tries')].mean()
    return nh_ave, h_nc_ave, h_ave

def first_pro(HW_df, df):
    ### Find the first part with hints for each student
    stu_list = df['username'].tolist()
    first_hint_pro = {}
    for ind, row in HW_df.iterrows():
        if not ind in stu_list:
            continue
        weekly_first = defaultdict(str)
        pro_done_time = {}
        col_count = 0
        hint_times = {}
        row_fine = row.fillna(-111)
        for r in row_fine[:-1]:
            title = cols[col_count]
            if int(title[0]) < 4 or r == -111:
                col_count += 1
                continue
            if type(r) is int:
                print r
            pro_done_time[title] = pd.to_datetime(r[-1][0])
            pro_hint_time = [pd.to_datetime(t[0]) for t in r if t[-1] == "Hint"]
            if pro_hint_time:
                first_pro_hint_time = min(pro_hint_time)
                if title[0] not in hint_times.keys():
                    hint_times[title[0]] = first_pro_hint_time
                elif hint_times[title[0]] > first_pro_hint_time:
                    hint_times[title[0]] = first_pro_hint_time
            col_count += 1
        
        if hint_times:
            for c in cols:
                if int(c[0]) < 4:
                    continue
                if c[0] in hint_times.keys() and c in pro_done_time and pro_done_time[c] > hint_times[c[0]]:
                    if not weekly_first[c[0]]:
                        weekly_first[c[0]] = [c[1]]
                    elif c[1] not in weekly_first[c[0]]:
                        weekly_first[c[0]].append(c[1])
        first_hint_pro[ind] = weekly_first
    return first_hint_pro

def sum_attempts(df, pros_tries, first_pros):
    ### Find sum of attempts for problems with hints.
    h_tries_sum = defaultdict(int)  #{stu_id:sum}
    h_pros = defaultdict(list)  #{stu_id:[pro_id, pro_id, ...]}

    ### Find sum of attempts for problems without hint.
    h_nc_tries_sum = defaultdict(int)
    h_nc_pros = defaultdict(list)
    
    h_nc_tries_weekly_sum = defaultdict(dict)
    h_nc_weekly_pros = defaultdict(dict)

    for ind, row in df.iterrows():
        single_h_nc_weekly_sum = defaultdict(int)
        single_h_nc_weekly_pros = defaultdict(list)
        for p in sorted(pros_tries):
            if int(p[0]) < 4 or int(p[0]) > 7:
                continue
            h_count = (p[0], p[1], 'hint_count')
            assign = (p[0],'assign')
            if row[assign] == '0':
                continue
            if first_pros[ind][p[0]]: #has at least one hint in assignment p[0]
                if row[h_count] > 0:
                    h_tries_sum[ind] += row[(p[0],p[1],'tries')]
                    h_pros[ind].append((p[0],p[1]))
                if p[1] in first_pros[ind][p[0]] and row[h_count] == 0:
                    h_nc_tries_sum[ind] += row[(p[0],p[1],'tries')]
                    h_nc_pros[ind].append((p[0],p[1]))
                    single_h_nc_weekly_sum[p[0]] += row[(p[0],p[1],'tries')]
                    single_h_nc_weekly_pros[p[0]].append((p[0],p[1]))
        for w in single_h_nc_weekly_sum:
            h_nc_tries_weekly_sum[w][ind] = single_h_nc_weekly_sum[w]
            h_nc_weekly_pros[w][ind] = single_h_nc_weekly_pros[w]
    return h_tries_sum, h_pros, h_nc_tries_sum, h_nc_pros, h_nc_tries_weekly_sum, h_nc_weekly_pros

def stats_between_(df_base, df, pros_df):
    diff_count = 0
    diff_sum = 0
    diff = np.array([])
    diff_n = []
    control_sum_tries = []
    treatment_sum_tries = []
    for name in pros_df:
        base_sum = 0
        for p in pros_df[name]:
            base_sum += df_base[p[0], p[1]]
        control_sum_tries.append(base_sum)
        treatment_sum_tries.append(df[name])
        diff = np.append(diff, base_sum - df[name])
        diff_n.append(name)
    ave = diff.mean()
    stat_values = stats.ttest_ind(control_sum_tries, treatment_sum_tries)
    #print diff_n[:10]
    #print diff[:10]
    return ave, stat_values[1], control_sum_tries, treatment_sum_tries, diff

def comparison(df_all, df, problems_list):   
    nh_ave, h_nc_ave, h_ave = find_ave(df_all)
    first_hint_pros = first_pro(HW_data, df_all)
    h_tries_sum, h_pros, \
    h_nc_tries_sum, h_nc_pros, \
    h_nc_tries_weekly_sum, h_nc_weekly_pros = sum_attempts(df, problems_list, first_hint_pros)

    ave, p_val, c, t, d = stats_between_(nh_ave, h_tries_sum, h_pros)
    #plot_cdf(d)
    print "Problems w/ hints"
    print "Baseline: no-hint group average"
    print "Baseline has ", ave, "more than hint group."
    print "P-val = ", p_val, "\n"
    
    ave, p_val, c, t, d = stats_between_(h_nc_ave, h_tries_sum, h_pros)
    #plot_cdf(d)
    print "Problems w/ hints"
    print "Baseline: no-show group average"
    print "Baseline has ", ave, "more than hint group."
    print "P-val = ", p_val, "\n"
    
    ave, p_val, c, t, d= stats_between_(nh_ave, h_nc_tries_sum, h_nc_pros)
    #plot_cdf(d)
    print "Problems w/o hints"
    print "Baseline: no-hint group average"
    print "Baseline has ", ave, " more than no-show group."
    print "P-val = ", p_val, "\n"
    
    ave, p_val, c, t, d = stats_between_(h_ave, h_nc_tries_sum, h_nc_pros)
    #plot_cdf(d)
    print "Problems w/o hints"
    print "Baseline: hint group average"
    print "Baseline has ", ave, "more than no-show group."
    print "P-val = ", p_val, "\n"

    
    '''for w in h_nc_tries_weekly_sum:
        ave, p_val, c, t, d= stats_between_(nh_ave, h_nc_tries_weekly_sum[w], h_nc_weekly_pros[w])
        print 'Week ', w
        print "(Problems w/o hint)Students in no-hint group has on average {0} more tries than".format(ave)
        print "students in hint group on problems without hint."
        print "P-val = ", p_val, "\n"'''
    

In [ ]:
HW_data = pkl.load(open('pkl/HW_df.pkl', 'rb'))
cols = HW_data.columns
HW_data['username'] = HW_data.index

In [ ]:
problems = list(set([(c[0],c[1]) for c in cols if len(c)==3]))

In [ ]:
assign_df = pd.read_pickle('assignment_table.pkl')
assign_df.columns

In [ ]:
features_df = pd.read_pickle('pkl/old_features_df.pkl')
features_df.columns

In [ ]:
#new_fea_df = pd.read_pickle('pkl/features_df.pkl')
#eval_info = pd.read_pickle('eval_info.pkl')
#hint_log = pd.read_pickle('hint_log.pkl')

In [ ]:
def build_hint_df(assign_df, features_df, hint_type):
    hints_df = df(HW_data['username'])
    h_all, h_non, h_uni, h_exp, h_con = get_hint_count()
    if hint_type == "all":
        hint_count = h_all
    elif hint_type == "none":
        hint_count = h_non
    elif hint_type == "conditional":
        hint_count = h_con
    elif hint_type == "universal":
        hint_count = h_uni
    elif hint_type == "expressions":
        hint_count = h_exp
    add_to_df('hint_count', hint_count, hints_df)
    add_assign(assign_df, hints_df)
    add_to_df('tries', get_tries(), hints_df)
    #add_to_df('time_diff', get_time_diff(), hints_df)
    #add_to_df('time_spent', get_time_spent(), hints_df)
    return hints_df

In [ ]:
hints_df_all = build_hint_df(assign_df, features_df, "all")

In [ ]:
pkl.dump(hints_df_all, open('hints_df_all.pkl', 'w'))

In [ ]:
hints_df = build_hint_df(assign_df, features_df, "all")
comparison(hints_df_all, hints_df, problems)

In [ ]:
hints_df = build_hint_df(assign_df, features_df, "none")
comparison(hints_df_all, hints_df, problems)

In [ ]:
hints_df = build_hint_df(assign_df, features_df, "conditional")
comparison(hints_df_all, hints_df, problems)

In [ ]:
hints_df = build_hint_df(assign_df, features_df, "universal")
comparison(hints_df_all, hints_df, problems)

In [ ]:
hints_df = build_hint_df(assign_df, features_df, "expressions")
comparison(hints_df_all, hints_df, problems)

In [ ]:
#### Average Final Scores
|Problems |no_hint|hint|P-value|
| ---|:-------:|:-------:|:-------:|
|with Hints|0.903/0.881|0.244/0.247|0.543|
|without Hint|0.780/0.897|0.219/0.171|0.0076|
|without Hint|0.846/0.924|0.269/0.182|0.024|


In [ ]:
### Almost all problems have hints.
### Students withno hints in some of the problem are students who didn't click show hint button.
### There are students who didn't click any show hint button in the whole week.
### Can't really distinguish no-click vs. not-assigned.

### Could study whether students tend to not click show-hint for same type of problems and finish them faster

In [ ]:
def fea_p_val_sum(fea_name, df):
    p_vals = defaultdict(float)
    h = defaultdict(float)
    nh = defaultdict(float)
    h_pro = defaultdict(int)
    nh_pro = defaultdict(int)
    for p in sorted(problems):
        if int(p[0]) < 4:
            continue
        pn = (p[0], p[1], 'hint_count')
        h_df = df[(df[pn] > 0)]
        nh_df = df[(df[pn] == 0) & (df[(p[0],'assign')] != '0')]
        if len(h_df) == 0 or len(nh_df) == 0:
            continue
        
        #print 'hint ', p, h_df[(p[0],p[1],fea_name)].mean()
        #print 'no hint ', p, nh_df[(p[0],p[1],fea_name)].mean()
        for ind,row in h_df.iterrows():
            h[ind] += row[(p[0],p[1],fea_name)]
            h_pro[ind] += 1
        for ind,row in nh_df.iterrows():
            nh[ind] += row[(p[0],p[1],fea_name)]
            nh_pro[ind] += 1
            
    nam_diff1 = [s for s in h.keys() if s not in nh.keys()]
    nam_diff2 = [s for s in nh.keys() if s not in h.keys()]
    for n in nam_diff1:
        del h[n]
    for n in nam_diff2:
        del nh[n]
    
    for k in h:
        h[k] = h[k]/h_pro[k]
    for k in nh:
        nh[k] = nh[k]/nh_pro[k]
    
    h_list = np.array(h.values())
    nh_list = np.array(nh.values())
    hint_mean = h_list.mean()
    nh_mean = nh_list.mean()
    hint_std = h_list.std()
    nh_std = nh_list.std()
    h_len = len(h_list)
    no_h_len = len(nh_list)
    diff = h_list - nh_list
    print diff.mean()
    
    #t_val, p_val = stats.ttest_ind_from_stats(hint_mean, hint_std, h, nh_mean, nh_std, no_h)
    t_val, p_val = stats.ttest_rel(h.values(), nh.values())
    return hint_mean, nh_mean, p_val

In [ ]:
hm, nhm, p = fea_p_val_sum('tries', hints_df_all)

In [ ]:
print hm
print nhm
print p

In [ ]:
fea_p_val_sum('tries', hints_df_all)

In [ ]:
p_vals, pros_time, h_time, nh_time = fea_p_val('time_spent', hints_df_all)
print len(pros_time)
###Print table for latex
for i in xrange(len(pros_time)):
    if p_vals[i] > 0.05:
        print pros_time[i], '&', p_vals[i]

In [ ]:
plt = fig_setting()
plot_hints(range(0,len(pros_time)), np.array(h_time)/3600., np.array(nh_time)/3600.)
plt.xticks(range(0,len(pros_time)), pros_time, rotation='vertical')
plt.title('Time Spent on Homework Problems')
plt.xlabel('Problem ID')
plt.ylabel('Hours Spent')
plt.legend(loc=2)
plt.savefig('times_analysis.png', bbox_inches='tight', pad_inches=0.2)

In [ ]:
### Output table for latex
p_vals_tries, pros_tries, h_tries, nh_tries = fea_p_val('tries', hints_df_all)
### Print table value for latex
for i in xrange(len(pros_time)):
    if p_vals_tries[i] > 0.05:
        print "{0} & {1:.3f} & {2:.3f} \\\\".format(pros_time[i], p_vals[i], p_vals_tries[i])

In [ ]:
plt = fig_setting()
plot_hints(range(0,len(pros_tries)), h_tries, nh_tries)
plt.xticks(range(0,len(pros_tries)), pros_tries, rotation='vertical')
plt.title('Number of Attempts', fontsize=BIGGER_SIZE)
plt.xlabel('Problem ID')
plt.ylabel('Number of Attempts')
plt.legend(loc=1)
plt.savefig('tries_analysis.png', bbox_inches='tight', pad_inches=0.2)

In [ ]:
### Number of attempts on problems without hints
### Look at students who got hint but did not clicked "show hint" to see whether they perform better
ave_nc, nc_p_val = stats_between_(h_nc_tries_sum, h_nc_pros, nh_ave, 'nh_ave_tries')
print "(Problems w/o hint)Students in no-hint group has on average {0} more tries than".format(ave_nc)
print "students who received hints but didn't click 'show hint'"
print "P-val = ", nc_p_val

In [ ]:
### A feature as students who click/notclick the hint.
### Assignment by assignment.

In [ ]:
### Number of attempts on problems without hint PER ASSIGNMENT.
h_na_diff_count_weekly = defaultdict(int)
h_na_diff_weekly = defaultdict(list)
h_na_control_sum_tries_weekly = defaultdict(list)
h_na_treatment_sum_tries_weekly = defaultdict(list)
for nam in h_na_pros:
    nh_sum = defaultdict(int)
    for p in h_na_pros[nam]:
        nh_sum[p[0]] += nh_ave[p[0], p[1], 'nh_ave_tries']
    h_na_control_sum_tries_weekly[p[0]].append(nh_sum[p[0]])
    h_na_treatment_sum_tries_weekly[p[0]].append(h_na_tries_sum_weekly[nam][p[0]])
    h_na_diff_weekly[p[0]].append(nh_sum[p[0]] - h_na_tries_sum_weekly[nam][p[0]])
    h_na_diff_count_weekly[p[0]] += 1
for w in ['5','6','7','8']:
    print sum(h_na_diff_weekly[w])/float(h_na_diff_count_weekly[w])
    print stats.ttest_rel(h_na_control_sum_tries_weekly[w], h_na_treatment_sum_tries_weekly[w])

In [ ]:
### Look at the universal hints see whether it helps - in HW and in final.

In [ ]:
### Questions in pdf (HW and Final)
### Students attempt without the correct solution
### Final score
### Make public dataset
### Clean up the code and setup a git.

In [ ]:
### Sort problems in the order students do it.
### Then filter out problems with hints.
### Put this result in abstract.
### Mention the weaker students but the data is not conclusive.

In [ ]:
nh_ave, h_nc_ave, h_ave = find_ave(hints_df)

In [ ]:
for k in nh_ave.keys():
    print nh_ave[k], h_nc_ave[k], h_ave[k]

In [ ]:
hints_df = build_hint_df(assign_df, features_df, "all")
p_vals_tries, pros_tries, h_tries, nh_tries = fea_p_val('tries', hints_df)

In [ ]:
#### alpha_hy - alpha_hn
#### Measure grade vs no-click

#### Do weekly on the problems with no hint and see whether p-value is small on specific assignment.



#### Plot p-value on the same graph!!!
#### Add time arrow on the graph of filtering problems.